Import the Relevant Packages

In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics
import tensorflow as tf
import tensorflow_datasets as tfds
import seaborn as sns
import datetime
import matplotlib.pyplot as plt

from tensorboard.plugins.hparams import api as hp
sns.set()

INFO:tensorflow:Enabling eager execution
INFO:tensorflow:Enabling v2 tensorshape
INFO:tensorflow:Enabling resource variables
INFO:tensorflow:Enabling tensor equality
INFO:tensorflow:Enabling control flow v2


# Downloading and preprocessing the MNIST dataset

In [2]:
# Before begining with the model and training, the dataset first needs to be preprocessed
# This is a very important step in all of machine learning

# The MNIST dataset is, in general, highly processed already - after all its 28x28 grayscale images of clearly visible digits
# Thus, the preprocessing will be limited to scaling the pixel values, shuffling the data and creating a validation set

# NOTE: When finally deploying a model in practice, it might be a good idea to include the prerpocessing as initial layers
# In that way, the users could just plug the data (images) directly, instead of being required to resize/rescale it before

### Defining some constants/hyperparameters

In [3]:
BUFFER_SIZE = 70_000 # for reshuffling
BATCH_SIZE = 128
NUM_EPOCHS = 20

## Downloading the MNIST dataset

In [4]:
mnist_dataset, mnist_info = tfds.load(name='mnist', with_info=True, as_supervised=True)

# When 'with_info' is set to True, tfds.load() returns two variables: 
# - the dataset (including the train and test sets) 
# - meta info regarding the dataset itself

### Extracting the train and test datasets

In [5]:
mnist_train, mnist_test = mnist_dataset['train'], mnist_dataset['test']

### Rescaling the images

Since each image is a gray-scale image with each pixel ranging from 0 to 255, it would be nice to rescale all of them to values ranging from 0 to 1 by simply dividing them by 255

In [6]:
def rescale_images(image, label):
    image = tf.cast(image, tf.float32) # Make sure all rescaled images will be of type float32
    image /= 255.0 # Achieve the scaling by dividing each image by 255.0
    return image, label

#### Scaling the Train and Test datasets

In [7]:
scaled_trained_and_validation_data = mnist_train.map(rescale_images) # Maps the old data to new scale
scaled_test_data = mnist_test.map(rescale_images) # Maps the old data to new scale

## Shuffling the Dataset

In [8]:
scaled_trained_and_validation_data = scaled_trained_and_validation_data.shuffle(BUFFER_SIZE)
scaled_test_data = scaled_test_data.shuffle(BUFFER_SIZE)

## Training and Validation Splitting
Splitting the now Shuffled scaled_trained_and_validation_data into training and validation datasets

In [9]:
# I am using train_test_split() from sklearn to split my dataset

# from sklearn.model_selection import train_test_split

In [10]:
# scaled_trained_data, scaled_validation_data = train_test_split(scaled_trained_and_validation_data,
#                                                               test_size=0.1,
#                                                               random_state=42)

Using train_test_split's  **test_size=0.1** does not return an integer value as size of the scaled_validation_data above so I will have to do the splitting manually

In [11]:
# Defining the size of the validation set
num_validation_samples = 0.1 * mnist_info.splits['train'].num_examples
num_validation_samples = tf.cast(num_validation_samples, tf.int64)

In [12]:
# Defining the size of the test set
num_test_samples = mnist_info.splits['test'].num_examples
num_test_samples = tf.cast(num_test_samples, tf.int64)

In [13]:
# Splitting the dataset into training + validation
train_data = scaled_trained_and_validation_data.skip(num_validation_samples)
validation_data = scaled_trained_and_validation_data.take(num_validation_samples)

## Batching the datasets

For proper functioning of the model, the batch size for the validation and test sets needs to be one big size that can take all the specific datasets into one batch


In [14]:
train_data = train_data.batch(BATCH_SIZE)
validation_data = validation_data.batch(num_validation_samples)
scaled_test_data = scaled_test_data.batch(num_test_samples)

In [15]:
# Extracting the numpy arrays from the validation data for the calculation of the Confusion Matrix
for images, labels in validation_data:
    images_val = images.numpy()
    labels_val = labels.numpy()

# Creating and Training the Model

### Defining Hyperparameters to be used in tunning

In [16]:
# Defining the hypermatarest we would test and their range
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([3,5,7]))
HP_OPTIMIZER = hp.HParam('optimizer', hp.Discrete(['adam', 'sgd']))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer('logs/hparam_tuning').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_OPTIMIZER],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

##  Cerate functions for training and logging purposes of the model

Outline of the model/architecture of the CNN to be implemented

CONV -> MAXPOOL -> CONV -> MAXPOOL -> FLATTEN -> DENSE

In [17]:
# Wrapping the model and training in a functions
def train_test_model(hparams):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(50, hparams[HP_FILTER_SIZE], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)), 
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(10)
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)

    # Compiling the model with parameter value for the optimizer
    model.compile(optimizer=hparams[HP_OPTIMIZER], loss=loss_fn, metrics=['accuracy'])
    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        train_data, 
        epochs = NUM_EPOCHS,
        callbacks = [early_stopping],
        validation_data = validation_data,
        verbose = 2
    )
    
    _, accuracy = model.evaluate(scaled_test_data)
    
    return accuracy

In [18]:
# Function to log the resuls
def run(log_dir, hparams):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

# Training the model with the different hyperparameters

In [19]:
# Performing a grid search on the hyperparameters needed for test
session_num = 0

for filter_size in HP_FILTER_SIZE.domain.values:
    for optimizer in HP_OPTIMIZER.domain.values:
    
        hparams = {
            HP_FILTER_SIZE: filter_size,
            HP_OPTIMIZER: optimizer
        }
        run_name = "run-%d" % session_num
        print('--- Starting trial: %s' % run_name)
        print({h.name: hparams[h] for h in hparams})
        run('logs/hparam_tuning/' + run_name, hparams)

        session_num += 1

--- Starting trial: run-0
{'filter_size': 3, 'optimizer': 'adam'}
Epoch 1/20
422/422 - 110s - loss: 0.2911 - accuracy: 0.9182 - val_loss: 0.0971 - val_accuracy: 0.9712
Epoch 2/20
422/422 - 107s - loss: 0.0794 - accuracy: 0.9763 - val_loss: 0.0672 - val_accuracy: 0.9788
Epoch 3/20
422/422 - 111s - loss: 0.0565 - accuracy: 0.9825 - val_loss: 0.0517 - val_accuracy: 0.9842
Epoch 4/20
422/422 - 116s - loss: 0.0475 - accuracy: 0.9852 - val_loss: 0.0479 - val_accuracy: 0.9860
Epoch 5/20
422/422 - 114s - loss: 0.0408 - accuracy: 0.9874 - val_loss: 0.0238 - val_accuracy: 0.9928
Epoch 6/20
422/422 - 118s - loss: 0.0352 - accuracy: 0.9894 - val_loss: 0.0244 - val_accuracy: 0.9937
Epoch 7/20
422/422 - 113s - loss: 0.0296 - accuracy: 0.9907 - val_loss: 0.0291 - val_accuracy: 0.9903
1/1 [==============================] - 5s 5s/step - loss: 0.0406 - accuracy: 0.9872
--- Starting trial: run-1
{'filter_size': 3, 'optimizer': 'sgd'}
Epoch 1/20
422/422 - 117s - loss: 1.3576 - accuracy: 0.6620 - val_loss:

#  Visualizing in Tensorboard

### Loading the Tensorboard extension

In [43]:
%load_ext tensorboard
%tensorboard --logdir "logs/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 7816), started 22:21:28 ago. (Use '!kill 7816' to kill it.)

In [45]:
%tensorboard --logdir logs/hparam_tuning/

Reusing TensorBoard on port 6006 (pid 8964), started 0:00:20 ago. (Use '!kill 8964' to kill it.)

From the Accuracy table, it can be seen that the Adam optimizers perform better.

Also, it the number of filters should not be too high or too small, it should just be at a midpoint.

In [41]:
from tensorboard import notebook
notebook.list() # View open TensorBoard instances

Known TensorBoard instances:
  - port 6006: logdir logs/hparam_tuning (started 3:38:41 ago; pid 11980)
  - port 6006: logdir {logs_base_dir} (started 3:40:56 ago; pid 13704)
  - port 6006: logdir logs/fit (started 1 day, 19:01:14 ago; pid 1608)
  - port 6006: logdir logs/hparam_tuning (started 17:47:13 ago; pid 7816)


To rerun tensorboard after the first run if there are any problems, open cmd and use the following commands

.../>taskkill /im tensorboard.exe /f

.../>del /q %TMP%\.tensorboard-info\*